## Estimating RNA velocity using scVelo and Seurat

Please ignore the code blocks with python code (grey background).

This dataset consisting of independent biological replicates, though the following analysis is carried out using the integrated dataset:

* RNA velocity estimation using the integrated datasets
    + assuming one consistent splicing kinetic across all cell types
    + evaluating if different splicing kinetics exist in different cell types (or in different biological replicates which could highlight batch effects impacting the RNA velocity estimations)

Note that currently the issue of removing batch effects across spliced and unspliced count matrices is an unsolved problem when analysing RNA velocities.

For the scVelo analysis approach we follow the [manual](https://scvelo.readthedocs.io/).

The key input into the RNA velocity analysis are two count matrices: one with pre-mature (unspliced) and one with mature (spliced) abundances. In this case these counts were obtained using the Alevin counting approach described [here](https://combine-lab.github.io/alevin-tutorial/2020/alevin-velocity/).
We chose alevin for counting as it was described in this [pre-print](https://www.biorxiv.org/content/10.1101/2020.03.13.990069v1.full).

We added the spliced and unspliced abundances estimated with Alevin to the Seurat object and used the spliced counts for integration with Seurat.

The input anndata object therefore contains:

1. data matrix `adata.X` contains the CellRanger identified counts (RNA assay information from the integrated Seurat object)
2. `adata.layers` contains the spliced and unspliced counts obtained from Alevin
3. `adata.obs` contains cell (observation) annotations
4. `adata.var` contains genes (variables) annotations
5. `adata.uns` contains unstructured annotations such as graphs



In [ ]:
import scvelo as scv
import pandas as pd

import os
import multiprocessing
import matplotlib as plt
import numpy as np

In [ ]:
# For beautified visualisations
scv.set_figure_params()

In [ ]:
# scVelo version used
scv.logging.print_version()

In [ ]:
# Get current working directory
fig_outs=os.path.join("..", "data", "derived_data", "Figures", "4-2.scVelo_WT_DAPT_remDblt")
data_out=os.path.join("..", "data", "derived_data", "python", "4-2.scVelo_WT_DAPT_remDblt")
tab_out=os.path.join("..", "data", "derived_data", "Tables", "4-2.scVelo_WT_DAPT_remDblt")

In [ ]:
os.makedirs(fig_outs, exist_ok=True)
os.makedirs(data_out, exist_ok=True)
os.makedirs(tab_out, exist_ok=True)

The adata object for this dataset looks as follows:

In [ ]:
adata = scv.read("../data/derived_data/R_objects/4-1.scVelo_prepare_data_remDblt/WT_DAPT.h5ad")
adata

The following cell annotations are present from the previous analysis in Seurat

In [ ]:
adata.obs

In [ ]:
adata.obs.cell_classification_res_1 = adata.obs.cell_classification_res_1.astype("category")

In [ ]:
adata.obs.cell_classification_res_1

The typical workflow in scVelo consists of subsequent calls of 

1. preprocessing tools (scv.pp.*)
2. analysis tools (scv.tl.*) 
3. plotting (scv.pl.*).

Below the proportions of spliced and unspliced counts are displayed. According to the [scVelo manual](https://scvelo.readthedocs.io/VelocityBasics.html), depending on the underlying used protocol, between 10%-25% of an unspliced proportion is typically detectable.

In [ ]:
scv.pl.proportions(adata, groupby = "cell_classification_res_1", figsize=(12,2))

Overall the unspliced proportion in these samples is with 20% within the expected range of the unspliced proportion.

# Basic Pre-processing

Basic pre-processing consists of gene selection and normalistion, followed by the calculation of first and second order moments (means and uncentered variances).

* gene selection by detection with a minimum number of counts and high variability (dispersion)
* normalising every cell by its total size and logarithmizing X
    * filtering and normalization is applied in the same vein to spliced/unspliced counts and X
    * logarithmizing is only applied to X
        * If X is already preprocessed from former analysis, it will not be touched

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000, enforce=True)

Require the first and second order moments (means and uncentred variances) among nearest neighbours in PCA space.

* first order is needed for deterministic velocity estimation 
* stochastic estimation also requires second order momnets

In [ ]:
scv.pp.moments(adata, n_pcs=25, n_neighbors=30)

Note: further preprocessing such as batch effect correction may be used, though the appropriate batch correction method is still an open question.

# Estimate RNA velocity

Velocities are vectors in gene expression space, obtained by solving a stochastic model of transcriptional dynamics. Other available modes (models) are:

* determinisitic:
    * using steady-state residuals
    * originally proposed by [La Manno et al. 2018](https://www.nature.com/articles/s41586-018-0414-6)
    * central assumption: common splicing rate across genes and full splicing dynamics with steady-state mRNA levels are observable
    * under the assumption that transcriptional phases (induction and repression) last sufficiently long to reach a steady-state equilibrium (active and inactive), velocities are quantified as the deviation of the observed ratio from its steady state ratio
        * equilibrium mRNA levels are approximated with a linear regression on the presumed steady states in the lower and upper quantiles
* stochastic:
    * aim: better capture the steady states
    * treating transcription, splicing and degradation as probabilistic events
* dynamical:
    * solving the full transcriptional dynamics of splicing kinetics using a likelihood-based dynamical model
    * infer gene-specific rates of transcription, splicing and degradation
    * recovers latent time of the underlying cellular processes
    * most powerful while computationally most expensive
    * aims to learn the unspliced/spliced phase trajectory
        * four transcriptional states are modeled to account for all possible configurations of gene activity
            * two dynamic transient states (induction and repression)
            * two steady states (active and inactive) reached after each dynamic transition
    * note that it still maintains the following assumptions:
        * constant gene-specific splicing and degradation rates 
        * two transcription rates each for induction and repression
            
    

Note that running the dynamical model requires prior running of `scv.tl.recover_dynamics(adata, **params)`.
The velocities are stored in `adata.layers` just as the count matrices.

In the following we use the generalized dynamical model to learn the full transcriptional dynamics of splicing kinetics.
It aims to learn the spliced/unspliced phase trajectory for each gene. It uses a likelihood-based expectation-maximisation framework to iteratively estimate the parameters of reaction rates and latent cell-specifc variables (e.g. transcriptional state and cell-internal latent time).

In [ ]:
coresNum = multiprocessing.cpu_count()
# efficient and robust estimation of velocities
scv.tl.recover_dynamics(adata, n_jobs=coresNum) # required for dynamical mode
scv.tl.velocity(adata, mode="dynamical", n_jobs=coresNum)

The velocities are projected into a lower-dimensional embedding (e.g. PCA or UMAP) by translating them into likely cell transitions.

The probabilities of one cell transitioning into another cell are computed using cosine correlation (between the potential cell transition and the velocity vector) and are stored in a matrix denoted as `velocity graph`.

In [ ]:
coresNum = multiprocessing.cpu_count()
scv.tl.velocity_graph(adata, n_jobs=coresNum)

In [ ]:
# This is required to solve the write issue with the h5ad below\n",
adata.__dict__['_raw'].__dict__['_var'] = adata.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})

In [ ]:
# store results for reuse
adata.write(os.path.join(data_out, "adata_dynamical_integrated.h5ad"), compression="gzip")

In [ ]:
viridis = plt.cm.get_cmap('plasma', 256)
newcolors = viridis(np.linspace(0, 1, len(adata.obs.cell_classification_res_1.cat.categories)), 0.5)

cell_states = ['13_Cycling Progenitors','5_Cycling Progenitors','1_Cycling Progenitors',
               '19_Cycling Progenitors','10_Cycling Progenitors','15_Cycling Progenitors',
               '17_Cycling Progenitors','18_Cycling Progenitors','3_Cycling Progenitors',
               '12_Cycling Progenitors','4_Cycling Progenitors','21_Cycling Progenitors',
               '0_ASCL1 Progenitors','2_ASCL1 Progenitors','7_ASCL1 Progenitors','11_Neurons',
               '14_Neurons','20_Neurons','16_Neurons','9_Neurons','6_Neurons','8_Neurons'] 
cell_states_colours = newcolors
cell_states_color_map = dict(zip(cell_states, cell_states_colours))
cell_states_color_map

# Visualisations

The velocities can be projected and visualised in any embedding.

In the first instance we plot the velocities at the single cell level, as grid lines and as stream lines using the PCA embedding:

In [ ]:
adata.obs.cell_classification_res_1 = adata.obs.cell_classification_res_1.cat.reorder_categories(['13_Cycling Progenitors','5_Cycling Progenitors','1_Cycling Progenitors',
               '19_Cycling Progenitors','10_Cycling Progenitors','15_Cycling Progenitors',
               '17_Cycling Progenitors','18_Cycling Progenitors','3_Cycling Progenitors',
               '12_Cycling Progenitors','4_Cycling Progenitors','21_Cycling Progenitors',
               '0_ASCL1 Progenitors','2_ASCL1 Progenitors','7_ASCL1 Progenitors','11_Neurons',
               '14_Neurons','20_Neurons','16_Neurons','9_Neurons','6_Neurons','8_Neurons'])

In [ ]:
adata.obs.group_cell_classification_res_1

In [ ]:
adata.obs.group_cell_classification_res_1 = adata.obs.group_cell_classification_res_1.replace([0,1,2],['Transitional Progenitors', 'Cycling Progenitors', 'Neurons'])

In [ ]:
scv.pl.velocity_embedding(adata, basis="pca", arrow_length=10, arrow_size=1, dpi=120, 
                          legend_loc='right margin', linewidth=0.02,
                         color="cell_classification_res_1", alpha=0.5, palette=cell_states_color_map, 
                         title="Single Cell Velocities in PCA space")

In [ ]:
scv.pl.velocity_embedding_grid(adata, basis='pca', color="cell_classification_res_1", 
                               arrow_length=5, 
                               legend_loc='right margin', palette=cell_states_color_map
                              )

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="pca", color="cell_classification_res_1",
                                 palette=cell_states_color_map,
                                 legend_loc='right margin', figsize=(10,6),
                                 save=os.path.join(fig_outs, "dynamical_integrated_pca_stream.png")
                                )

The first two PCAs seem to visualise the progression from more immature precurser cells (Cycling progenitors) towards more mature neurons via ASCL1 progenitors. This is the expected direction of development within this experiment.

We can also visualise the RNA velocities using the UMAP embeddings:

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="umap", color="cell_classification_res_1", legend_loc='right margin', 
                                save=os.path.join(fig_outs, "dynamical_integrated_umap_cell_states_stream.png"), 
                                palette=cell_states_color_map,
                                title="Velocity stream in UMAP space", figsize=(10,6))

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="umap", color="group_cell_classification_res_1", legend_loc='right margin', 
                                save=os.path.join(fig_outs, "dynamical_integrated_umap_group_cell_states_stream.png"), 

                                title="Velocity stream in UMAP space", figsize=(10,6))


To ensure that the observed RNA velocities do not overlap with cells stemming from just one of the integrated samples, we visualised the different samples:

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="umap", color="orig_ident", 
                                 legend_loc='right margin', 
                                figsize=(10,6),
                                 save=os.path.join(fig_outs, "dynamical_integrated_umap_orig_ident_stream.png")
                                )

Finally we visualised the expression of some known marker genes (using the "RNA" layer, i.e. default `adata.X`).
Using the following markers: ASCL1 (ASCL1 progenitors), STMN2 and NEFM (Neurons), MKI67 and FABP7 (Cycling progenitors).

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="umap", color="ASCL1", legend_loc='right margin',
                                 figsize=(10,6),
                                 use_raw=False, color_map = "Reds")

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="umap", color="STMN2", legend_loc='right margin', 
                                 figsize=(10,6),
                                 use_raw=False, color_map = "Reds")

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="umap", color="NEFM", legend_loc='right margin',
                                 figsize=(10,6),
                                 use_raw=False, color_map = "Reds")

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="umap", color="MKI67", legend_loc='right margin', 
                                 figsize=(10,6),
                                 use_raw=False, color_map = "Reds")

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="umap", color="FABP7", legend_loc='right margin',
                                 figsize=(10,6),
                                 use_raw=False, color_map = "Reds")

# Interpret Velocities

Examine individual gene dynamis via phase portraits to understand how inferred directions are supported by particular genes.

In the phase portraits, the black line corresponds to the estimated "steady-state" ratio, i.e. the ratio of unspliced to spliced mRNA abundance which is in a constant transcriptional state.

RNA velocity for a particular gene is determined as the residual, i.e. how much an observation deviates from that steady-state line.

In [ ]:
scv.pl.velocity(adata, ['ASCL1', 'STMN2', 'NEFM', 'MKI67', 'FABP7'], ncols=1, basis="umap")

In [ ]:
scv.pl.scatter(adata, 'STMN2', color=['cell_classification_res_1', 'velocity'], palette=cell_states_color_map,
                                 legend_loc='right margin', wspace=1, figsize=(9,5))

Systematic identification of genes that may help explain the resulting vector field and inferred lineages. For this, we test which genes have cluster-specific differential velocity expression (being significantly higher/lower compared to the remaining population).
`scv.tl.rank_velocity_genes` runs a differential expression test (Welch t-test with overestimated variance to be conservative) on velocity expression to find genes in a cluster that show dynamics that is transcriptionally regulated differently compared to all other clusters (e.g. induction in that cluster and homeostasis in the remaining population). 
It may be worth considering to filter based on `min_likelihood` in addition to `min_corr`? Note that the top 100 genes are shown for each cluster.

In [ ]:
scv.tl.rank_velocity_genes(adata, groupby='cell_classification_res_1', min_corr=.3)

df_diff_velo = scv.DataFrame(adata.uns['rank_velocity_genes']['names'])
df_diff_velo.head()

In [ ]:
scv.pl.scatter(adata, 'NNAT', color=['cell_classification_res_1', 'velocity'], palette=cell_states_color_map,
                                 legend_loc='right margin', wspace=1, figsize=(10, 6))

Below we show the top three ranked differential velocity genes for each identified cell state.

In [ ]:
for cell_state in adata.obs.cell_classification_res_1.cat.categories:
   scv.pl.scatter(adata, df_diff_velo[cell_state][:3], ylabel=cell_state, color=["cell_classification_res_1"], 
                  palette=cell_states_color_map,
                                 legend_loc='none',
                  frameon=False, size=20, linewidth=1.5)

The excel file `scVelo_differential_velocity_top100_corr0.3.xlsx` contains the top 100 velocity genes with a minimum correlation of 0.3.

In [ ]:
df_diff_velo.to_excel(os.path.join(tab_out, "scVelo_differential_velocity_top100_corr0.3.xlsx"))

# Velocities in cycling progenitors



Cell cycle detected by RNA velocity (circles in the velocity stream plot), can be confirmed by visualising the cell cycle scores (calculated in Seurat).

In [ ]:
scv.pl.scatter(adata, color_gradients=['S_Score', 'G2M_Score'], smooth=True, perc=[5, 95])

We may screen through S and G2M phase markers. The `scv.tl.rank_velocity_genes` module calculated Spearman correlation scores, which can be used to rank/sort the cell cycle phase marker genes to subsequently show their phase portraits.

In [ ]:
s_genes, g2m_genes = scv.utils.get_phase_marker_genes(adata)

In [ ]:
s_genes = scv.get_df(adata[:, s_genes], 'spearmans_score', sort_values=True).index
g2m_genes = scv.get_df(adata[:, g2m_genes], 'spearmans_score', sort_values=True).index


In [ ]:
kwargs = dict(frameon=False, ylabel='cell cycle genes', color="Phase")
scv.pl.scatter(adata, list(s_genes[:2]) + list(g2m_genes[:3]), **kwargs)

In [ ]:
scv.pl.velocity(adata, ['HELLS', 'ATAD2', 'CDCA3'], ncols=1, add_outline=True, color="Phase")

# Speed and coherence

The speed or rate of differentiation is given by the length of the velocity vector.
In addition the coherence of the vector field (i.e. how a velocity vector correlates with its neighboring velocities) provides a measure of confidence.

In [ ]:
scv.tl.velocity_confidence(adata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adata, c=keys, cmap='Reds', perc=[5, 95])

We can use velocity length to gain insights into where cells differentiate at a faster (slower) pace, e.g. areas associated with S phase progenitors seem to be associated with a faster differntiation pace. Furthermore, low velocity confidence indicates regions in which the direction is undetermined.

On a cluster level we can observe that differentiation speed is particulaly fast in S phase cycling progenitors & Neurons, while other clusters show slower differentiation rate. 

In [ ]:
df = adata.obs.groupby('cell_classification_res_1')[keys].mean().T
df.style.background_gradient(cmap='coolwarm', axis=1)

# Velocity graph and pseudotime

We can visualise the velocity graph to portray all velocity-inferred cell-to-cell connections/transitions. (Confining it to high-probability transitions by setting a `threshold`).

In [ ]:
scv.pl.velocity_graph(adata, threshold=.5, basis="umap", color="cell_classification_res_1", figsize=(10,8),
                      edge_width=.1, palette=cell_states_color_map,
                      legend_loc='right margin', title="Velocity-inferred cell-to-cell transitions")

The graph can be used to draw descendents/ancestors coming from a specified cell. 

Here a ASCL1 progenitor cell is traced to its potential fate.

In [ ]:
adata.obs.index.get_loc(adata.obs[adata.obs.cell_classification_res_1 == "7_ASCL1 Progenitors"].index[1])

In [ ]:
x, y = scv.utils.get_cell_transitions(adata, basis='umap', starting_cell=1291)
ax = scv.pl.velocity_graph(adata, c='lightgrey', edge_width=.05, show=False, threshold=.5)
ax = scv.pl.scatter(adata, x=x, y=y, s=120, c='ascending', cmap='gnuplot', ax=ax)

Here a Cycling_progenitors_cluster_12 cell is traced to its potential fate.

In [ ]:
adata.obs.index.get_loc(adata.obs[adata.obs.cell_classification_res_1 == "12_Cycling Progenitors"].index[1])

In [ ]:
x, y = scv.utils.get_cell_transitions(adata, basis='umap', starting_cell=514)
ax = scv.pl.velocity_graph(adata, c='lightgrey', edge_width=.05, show=False, threshold=.5)
ax = scv.pl.scatter(adata, x=x, y=y, s=120, c='ascending', cmap='gnuplot', ax=ax)

Based on the velocity graph, a velocity pseudotime can be computed. After inferring a distribution over root cells from the graph, it measures the average number of stpes it takes to reach a cell after walking along the graph starting from the root cells.

It implicitly infers the root cells (in contrast to diffusion pseudotime) and is based on the directed velocity graph (rather than similarity-based diffusion kernel). 

In [ ]:
scv.tl.velocity_pseudotime(adata)
scv.pl.scatter(adata, color='velocity_pseudotime', cmap='gnuplot', 
               title="Velocity-inferred Pseudotime", figsize=(10,8))

# Kinetic rate parameters

The rates of RNA transcription, splicing and degradation are estimated (in the dynamical model) without the need of any experimental data. They can be used to better understand the cell identity and phenotypic heterogeneity.

In [ ]:
df = adata.var
df = df[(df['fit_likelihood'] > .1) & df['velocity_genes'] == True]

kwargs = dict(xscale='log', fontsize=16)
with scv.GridSpec(ncols=3) as pl:
    pl.hist(df['fit_alpha'], xlabel='transcription rate', **kwargs)
    pl.hist(df['fit_beta'] * df['fit_scaling'], xlabel='splicing rate', xticks=[.1, .4, 1], **kwargs)
    pl.hist(df['fit_gamma'], xlabel='degradation rate', xticks=[.1, .4, 1], **kwargs)

scv.get_df(adata, 'fit*', dropna=True).head()



The estimated gene-specific parameters:

* rates of
    * transcription (`fit_alpha`)
    * splicing (`fit_beta`)
    * degradation (`fit_gamma`)
* switchig time point (`fit_t_`)
* a scaling parameter to adjust for underrepresented unspliced reads (`fit_scaling`)
* standard deviation of unspliced and spliced reads (`fit_std_u`, `fit_std_s`)
* gene likelihood (`fit_likelihood`)
* inferred steady-state levels (`fit_steady_u`, `fit_steady_s`) with their corresponding p-values (`fit_pval_steady_u`, `fit_pval_steady_s`)
* overall model variance (`fit_variance`)
* scaling factor to align the gene-wse latent times to a universal, gene-shared latent time (`fit_alignment_scaling`)


# Latent time

The dynamical model recovers the latent time of the underlying cellular processes. This latent time represents the cell's internal clock and approximates the real time experienced by cells as they differentiate (based only on the transcriptional dynamics).

In [ ]:
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot',
               title="Velocity inferred Latent Time", figsize=(10,8))

In [ ]:
df = adata.obs
kwargs = dict(xscale='log', fontsize=16)

with scv.GridSpec(ncols=2) as pl:
    pl.hist(df['velocity_confidence'], xlabel='velocity_confidence', xticks=[.7, .9, 1], **kwargs)
    pl.hist(adata.var['gene_count_corr'], xlabel='gene_count_corr', xticks=[.1, .4, 1],**kwargs)
    


In [ ]:
top_genes = adata.var["fit_likelihood"].sort_values(ascending=False).index[:300]
scv.pl.heatmap(adata, var_names=top_genes, sortby="latent_time", col_color="cell_classification_res_1", 
               n_convolve=100, figsize=(10,8), palette=cell_states_color_map, 
               save=os.path.join(fig_outs, "heatmap_top300_driver_genes.png"))

The heatmap shows the top 300 genes (based on `fit_likelihood`) and sorted according to their inferred latent time. 

In [ ]:
marker_genes = ['HES5', 'FABP7', 'MCM2', 'MCM6', 'VIM', 'NES', 'MKI67', 'TOP2A', 'ASCL1', 'SOX4', 'DLL1', 'GADD45G', 'HES6', 'IGFBP5', 'CDKN1C', 'DLL3', 'RGS16', 'DLX2', 'DLX5', 'ELAVL4', 'STMN2', 'GAD2', 'SLC17A6', 'MAP2']

                      
scv.pl.heatmap(adata, var_names=marker_genes,
               sortby="latent_time", col_color="cell_classification_res_1", 
               sort=False,
               n_convolve=100, figsize=(10,8), palette=cell_states_color_map,
               save=os.path.join(fig_outs, "heatmap_driver_genes_of_interest.png"))

In [ ]:
marker_genes = ['MKI67', 'MCM2', 'MCM6', 'TOP2A', 'HES5', 'FABP7', 'VIM', 'NES', 'ASCL1', 'SOX4', 'DLL1', 'GADD45G', 'CDKN1C', 'HES6', 'DLL3', 'RGS16', 'IGFBP5', 'DLX2', 'DLX5', 'MAP2', 'ELAVL4', 'STMN2', 'GAD2', 'SLC17A6']
                              
scv.pl.heatmap(adata, var_names=marker_genes,
               sortby="latent_time", col_color="group_cell_classification_res_0_8", 
               sort=False,
               n_convolve=100, figsize=(10,8), 
               save=os.path.join(fig_outs, "heatmap_driver_genes_of_interest_group_cell.png"))

The heatmap shows the provided marker genes (based on `fit_likelihood`) and sorted according to their inferred latent time. 

# Top-likelihood genes

Driver genes display pronounced dynamic behaviour and are systematically detected via their characterization by high likelihoods in the dynamic model.

In [ ]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index
scv.pl.scatter(adata, basis=top_genes[:12], ncols=4, frameon=False, color="cell_classification_res_1",
              palette=cell_states_color_map
              )

In [ ]:
var_names = ['STMN2', 'TCF4', 'NREP']
scv.pl.scatter(adata, var_names, frameon=False, color="cell_classification_res_1",
               palette=cell_states_color_map,
               ncols = 3)
scv.pl.scatter(adata, x='latent_time', y=var_names, frameon=True, color="cell_classification_res_1", layer="RNA")

The above shows the phase portraits and the RNA expression of three example driver genes.

In [ ]:
driver_genes = adata.var.dropna(subset = ['fit_likelihood'])['fit_likelihood'].sort_values(ascending=False)

The excel file `scVelo_driver_genes_complete_dataset.xlsx` contains all driver genes with a fit_likelihood > 0.

In [ ]:
driver_genes.to_excel(os.path.join(tab_out, "scVelo_driver_genes_complete_dataset.xlsx"))

In [ ]:
driver_genes

In [ ]:
pl.hist(driver_genes, yscale='log', bins=50)

In [ ]:
# Based on the hisotgram it may be sensible to select top likelihood genes as those with a fit_likelihood > 0.3
driver_genes[driver_genes > 0.3]

The excel file `scVelo_driver_genes_complete_dataset_likelihood0.3.xlsx` contains the driver (top-likelihood genes) with a minimum likelihood > 0.3.

In [ ]:
driver_genes[driver_genes > 0.3].to_excel(os.path.join(tab_out, "scVelo_driver_genes_complete_dataset_likelihood0.3.xlsx"))

# Cluster-specific top-likelihood genes

Partial gene likelihoods can be computed for each cluster of cells to enable cluster-specific identification of potential drivers

In [ ]:
scv.tl.rank_dynamical_genes(adata, groupby='cell_classification_res_1')
df_cluster_driver = scv.get_df(adata, 'rank_dynamical_genes/names')
df_cluster_driver.head(5)

The full table of the top 100 driver genes per cluster can be found in the excel file `scVelo_driver_genes_for_each_cluster_top100.xlsx`

In [ ]:
df_cluster_driver.to_excel(os.path.join(tab_out, "scVelo_driver_genes_for_each_cluster_top100.xlsx"))

In [ ]:
dyn_genes_result = adata.uns['rank_dynamical_genes']
groups = dyn_genes_result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key: dyn_genes_result[key][group]
    for group in groups for key in ['names', 'scores']}).head(5)

For each of the cluster-specific driver genes we filtered the genes for those with a score above 0.3 and write the genes for each cluster to an excel file.

In [ ]:
for group in groups:
    group_driver = pd.DataFrame({key: dyn_genes_result[key][group]
                                 for key in ['names', 'scores']})
    group_driver_f = group_driver[group_driver.scores > 0.3]
    group_driver_f.to_excel(os.path.join(tab_out, "scVelo_driver_genes_for_" + 
                                                     group.replace(" ", "_").replace("/", "") + 
                                                     "_score_0.3.xlsx"))
   # print(group_driver_f)

Note that some cluster-specific driver genes are shared between clusters, e.g. QKI in most clusters.

In [ ]:
for cell_state in adata.obs.cell_classification_res_1.cat.categories:
    vars=df_cluster_driver[cell_state][:3]
    vars=vars.tolist()
    scv.pl.scatter(adata, vars, ylabel=cell_state, color=["cell_classification_res_1"], frameon=False)
    scv.pl.scatter(adata, x='latent_time', y=vars, frameon=True, color="cell_classification_res_1", layer="RNA")
   

# PAGA velocity graph

## Cell state grouped

[PAGA](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1663-x) graph abstraction has benchmarked as top-performing method for trajectory inference. It provides a graph-like map of the data topology with weighted edges corresponding to the connectivity between two clusters. Here, PAGA is extended by velocity-inferred directionality.

In [ ]:
scv.tl.paga(adata, groups='group_cell_classification_res_1')

In [ ]:
df = scv.get_df(adata, 'paga/transitions_confidence', #precision=2, 
                columns='group_cell_classification_res_1', index = 'group_cell_classification_res_1').T
df.style.background_gradient(cmap='Blues').format('{:.2g}')


The above table can be visualised using a directed graph superimposed on the UMAP embedding.

In [ ]:
scv.pl.paga(adata, basis='umap', size=50, alpha=.1, figsize=(10,6),
            min_edge_width=2, node_size_scale=1.5)

Cycling progenitors (cluster1) seem to transition into ASCL1 progenitos which further differentiate into neurons. There are also a range of transitions between the different cycling neuron progenitor clusters.

In [ ]:
scv.pl.paga(adata, basis='pca', size=50, alpha=.1, figsize=(10,6),
            min_edge_width=2, node_size_scale=1.5)

## Cell state within clusters

In [ ]:
scv.tl.paga(adata, groups='cell_classification_res_1')

In [ ]:
df = scv.get_df(adata, 'paga/transitions_confidence', #precision=2, 
                columns='cell_classification_res_1', index = 'cell_classification_res_1').T
df.style.background_gradient(cmap='Blues').format('{:.2g}')


The above table can be visualised using a directed graph superimposed on the UMAP embedding.

In [ ]:
scv.pl.paga(adata, basis='umap', size=50, alpha=.1, figsize=(10,6),
            min_edge_width=2, node_size_scale=1.5, 
            save=os.path.join(fig_outs, "dynamical_integrated_umap_cell_paga_velocity_graph.png"))

Cycling progenitors (cluster1) seem to transition into ASCL1 progenitos which further differentiate into neurons. There are also a range of transitions between the different cycling neuron progenitor clusters.

In [ ]:
scv.pl.paga(adata, basis='pca', size=50, alpha=.1, figsize=(10,6),
            min_edge_width=2, node_size_scale=1.5)

In [ ]:
# store results for reuse
adata.write(os.path.join(data_out, "adata_dynamical_integrated.h5ad"), compression="gzip")
# adata = scv.read(os.path.join(data_out, "adata_dynamical_integrated.h5ad"))

# Differential Kinetics

Systems with multiple lineages and processes have genes that likely show different kinetic regimes across subpopulations. Distinct cell states and lineages are typically governed by different variations in the gene regulatory networks and may hence exhibit different splicing kinetics. This gives rise to genes that display multiple trajectories in phase space.

The dynamical model can be used to perform a likelihood-ratio test for differential kinetics. This allows the detection of clusters that display kinetic behaviour that connot be well explained by a single model of the overall dynamics. Clustering cell types into their different kinetic regimes then allows fitting each regime separately.

## Differential Kinetic test

Distinct cell types and lineages may exhibit different kinetic regimes that are governed by a different network structure. Even in related cell types or lineages, kinetics may be differential due to alternative splicing, alternative polyadenylation and modulations in RNA degradation.

Each cell type is tested whether an independent fit yields a significantly improved likelihood.
The likelihood ratio, following an asymptotic chi-squared distribution can be tested for significance. For efficiency reasons, the default uses an orthogonal regression instead of a full phase trajectory to test whether a cluster is well explained by the overall kinetics or exhibits a different kinetic.

In [ ]:
scv.tl.differential_kinetic_test(adata, groupby='group_cell_classification_res_1')


In [ ]:
diff_kinetics=scv.get_df(adata[:, ], ['fit_diff_kinetics', 'fit_pval_kinetics'])
diff_kinetics.sort_values(by = ["fit_pval_kinetics"], ascending=True)

In this case over 460 genes shows differential splicing kinetics across clusters.

In [ ]:
var_names=["DNAJC15", "ONECUT1", "WLS"]
kwargs = dict(linewidth=2, add_linfit=True, frameon=False)
scv.pl.scatter(adata, basis=var_names, add_outline='fit_diff_kinetics', **kwargs)

## Recompute Velocities

Based on the computed differential kinetics we can recompute the estimated RNA velocities.

In [ ]:
coresNum = multiprocessing.cpu_count()
scv.tl.velocity(adata, diff_kinetics=True, mode="dynamical" )
scv.tl.velocity_graph(adata, n_jobs=coresNum)

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="umap", color="group_cell_classification_res_1", 
                                 
                                 legend_loc='right margin', figsize=(10,6), 
                                 save=os.path.join(fig_outs, "dynamical_integrated_umap_stream_differential_kinetics.png"),
                                title="Velocity estimates assuming differential kinetics")

In [ ]:
marker_genes = ['MKI67', 'MCM2', 'MCM6', 'TOP2A', 'HES5', 'FABP7', 'VIM', 'NES', 'ASCL1', 'SOX4', 'DLL1', 'GADD45G', 'CDKN1C', 'HES6', 'DLL3', 'RGS16', 'IGFBP5', 'DLX2', 'DLX5', 'MAP2', 'ELAVL4', 'STMN2', 'GAD2', 'SLC17A6']
                              
scv.pl.heatmap(adata, var_names=marker_genes,
               sortby="latent_time", col_color="group_cell_classification_res_1", 
               sort=False,
               n_convolve=100, figsize=(10,8), 
               save=os.path.join(fig_outs, "heatmap_driver_genes_of_interest_group_cell.png"))

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="pca", color="cell_classification_res_1", 
                                 legend_loc='right margin', figsize=(10,6), palette=cell_states_color_map,
                                 save=os.path.join(fig_outs, "dynamical_pca_stream_differential_kinetics.png"))

In this case, assuming differential splicing kinetics between clusters did not significantly change the overall velocity streams and predicted developmental trajectories. Using the expression of the marker genes and sorting cells according to their latent time (see heatmap above) might be better confirmed to the experimental expectations without using the differential kinetics assumptions, i.e. with used the initial RNA velocity estimates for the analysis